# 生成数据 

In [1]:
from utils.TransPose.preprocess import process_amass, process_dipimu

print("训练时候需要对加速度计数据添加重力。旋转矩阵添加欧拉角的噪声，加速度计也添加对应的噪声，增加泛化性能")
process_amass()
# process_dipimu() 

训练时候需要对加速度计数据添加重力。旋转矩阵添加欧拉角的噪声，加速度计也添加对应的噪声，增加泛化性能
shape.pt: SMPL parameter(没什么关系)
 pose.pt: 24个关节姿态的轴角表示
joint.pt: 24个关节相对位置
 tran.pt: 人物动作的绝对位置(与绝对位置结合可得真实位置)
 vrot.pt: 生成的陀螺数据(右脚尖，左脚尖，后腰)，实际上是姿态旋转矩阵，需要AHRS算出姿态
 vacc.pt: 生成的n系加速度计数据(右脚尖，左脚尖，后腰)，需要根据姿态旋转到n系东北天
有用的关节：     11,     10,     8,     7,      5,     4,       2,       1
    对应为： 右脚尖， 左脚尖；右脚踝，左脚踝；右膝盖，左膝盖；右大腿根，左大腿根
##################################################
Reading HumanEva


0it [00:00, ?it/s]

Reading MPI_HDM05


0it [00:00, ?it/s]

Reading SFU


...

Reading MPI_Limits


0it [00:00, ?it/s]

Reading DFaust67


0it [00:00, ?it/s]

AssertionError: AMASS dataset not found. Check config.py or comment the function process_amass()

In [2]:
import yaml, argparse
from utils.wmx_tools import create_parser

# 导入创建的yaml文件
with open('./configs/train_params_LSTM.yaml', 'r') as f:
    # yml = yaml.load(f, yaml.FullLoader)
    
    parser = argparse.ArgumentParser()
    parser.set_defaults(**yaml.load(f, yaml.FullLoader))
    args = parser.parse_args(args=[])

In [12]:
from sklearn.model_selection import train_test_split
from utils.wmx_tools import normalize_and_concat_X, normalize_and_concat_y
import torch, os
from tqdm.notebook import tqdm
from utils.TransPose.config import paths, joint_set, acc_scale, vel_scale

out_pose = torch.load(os.path.join(paths.amass_dir, 'pose.pt'))      # 24个关节的轴角表示
out_tran = torch.load(os.path.join(paths.amass_dir, 'tran.pt'))      # 人物动作的绝对位置
out_joint = torch.load(os.path.join(paths.amass_dir, 'joint.pt'))     # 24个关节相对位置
out_vrot = torch.load(os.path.join(paths.amass_dir, 'vrot.pt'))      # 生成的陀螺数据(实际上是姿态旋转矩阵，需要AHRS算出姿态)
out_vacc = torch.load(os.path.join(paths.amass_dir, 'vacc.pt'))      # 生成的加速度计数据
out_shape = torch.load(os.path.join(paths.amass_dir, 'shape.pt'))     # SMPL parameter(没什么关系)

print(f"out_tran的形状为,{out_tran[1].shape}")
print(f"out_joint的形状为,{out_joint[0].shape}")
# 检查列表长度和第一个元素的形状
print(f"out_vacc 样本数量: {len(out_vacc[0])}")
if len(out_vacc) > 0:
    print(f"第一个样本的加速度数据形状: {out_vacc[0].shape}")

print(f"\nout_vrot 样本数量: {len(out_vrot)}")
if len(out_vrot) > 0:
    print(f"第一个样本的旋转数据形状: {out_vrot[0].shape}")

X, y = [], []
velocity_list = []
dataacc = []
for i in tqdm(range(len(out_tran))):
    #加速度数据标准化并归一化
    if (out_vacc[i].shape[0] > 100):  #超过30个数据再归一化
        X_tuple = normalize_and_concat_X(out_vacc[i] / acc_scale, out_vrot[i], False)
        X.append(X_tuple)
        # X.append(normalize_and_concat_X(out_vacc[i] / acc_scale, out_vrot[i], False))     
        # y.append(normalize_and_concat_y(out_tran[i], out_pose[i], out_joint[i], out_vrot[i], out_shape[i]))
        y_tuple = normalize_and_concat_y(out_tran[i], out_pose[i], out_joint[i], out_vrot[i], out_shape[i])
        y.append(y_tuple)
        data =X_tuple
        dataacc.append(data)
        velocity = y_tuple[3]
        velocity_list.append(velocity)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=857857)

/tmp/ipykernel_56474/3052335532.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  out_pose = torch.load(os.path.join(paths.amass_dir, 'pose.pt'))      # 24个关节的轴角表示
/tmp/ip

out_tran的形状为,torch.Size([1460, 3])
out_joint的形状为,torch.Size([1677, 24, 3])
out_vacc 样本数量: 1677
第一个样本的加速度数据形状: torch.Size([1677, 3, 3])

out_vrot 样本数量: 9124
第一个样本的旋转数据形状: torch.Size([1677, 3, 3, 3])


  0%|          | 0/9124 [00:00<?, ?it/s]

In [11]:
print(len(X))

8839


In [8]:
import pandas as pd
import numpy as np

def save_velocity_to_csv(velocity_list, output_path="velocity_data.csv"):
    # 收集所有速度数据
    all_velocity = []
    for sample_idx, velocity_tensor in enumerate(velocity_list):
        # 转换张量为numpy格式
        velocity_np = velocity_tensor.cpu().numpy()  # 如果张量在GPU上
        
        # 确保形状为 [T, 3]（时间步长 x 速度分量）
        if len(velocity_np.shape) == 1:
            velocity_np = velocity_np.reshape(-1, 3)
            
        # 为每个时间步添加样本ID和时间步ID
        for t in range(velocity_np.shape[0]):
            all_velocity.append([
                sample_idx,  # 样本ID
                t,           # 时间步ID
                velocity_np[t, 0],  # 速度X分量
                velocity_np[t, 1],  # 速度Y分量
                velocity_np[t, 2]   # 速度Z分量
            ])
    
    # 创建DataFrame
    df = pd.DataFrame(
        all_velocity,
        columns=["SampleID", "Timestep", "Velocity_X", "Velocity_Y", "Velocity_Z"]
    )
    
    # 保存到CSV
    df.to_csv(output_path, index=False)
    print(f"速度数据已保存至: {output_path}")

# 调用函数
save_velocity_to_csv(velocity_list)

速度数据已保存至: velocity_data.csv


In [ ]:
from sklearn.model_selection import train_test_split
from utils.wmx_tools import normalize_and_concat_X, normalize_and_concat_y
import torch, os
from tqdm.notebook import tqdm
from utils.TransPose.config import paths, joint_set, acc_scale, vel_scale

print(f"out_tran的形状为,{out_tran[1].shape}")
print(f"out_joint的形状为,{out_joint[0].shape}")
# 检查列表长度和第一个元素的形状
print(f"out_vacc 样本数量: {len(out_vacc)}")
print(f"out_vacc 样本数量: {out_vacc[0].shape[0]}")
if len(out_vacc) > 0:
    print(f"第一个样本的加速度数据形状: {out_vacc[0].shape}")

print(f"\nout_vrot 样本数量: {len(out_vrot)}")
if len(out_vrot) > 0:
    print(f"第一个样本的旋转数据形状: {out_vrot[0].shape}")

out_tran的形状为,torch.Size([1460, 3])
out_joint的形状为,torch.Size([1677, 24, 3])
out_vacc 样本数量: 9124
out_vacc 样本数量: 1677
第一个样本的加速度数据形状: torch.Size([1677, 3, 3])

out_vrot 样本数量: 9124
第一个样本的旋转数据形状: torch.Size([1677, 3, 3, 3])


In [ ]:
torch.save({"X": X_train, 'y': y_train},'./data/data_train_wmx100.pt')
torch.save({"X": X_val, 'y': y_val},'./data/data_val_wmx100.pt')


# # y: [temp_local_full_joint_position, temp_bone_length, temp_contact_probability, temp_velocity]
# torch.save({"X": torch.cat([X_train[i].unfold(dimension = 0, size = args.window_size, step = args.window_step).permute(0, 2, 1) for i in range(len(X_train))], dim = 0), 
#         'y': [torch.cat([y_train[i][0].unfold(dimension = 0, size = args.window_size, step = args.window_step).permute(0, 3, 1, 2) for i in range(len(y_train))]), 
#             torch.cat([y_train[i][1].squeeze(1).repeat(y_train[i][0].shape[0], 1).unfold(dimension = 0, size = args.window_size, step = args.window_step).permute(0, 2, 1) for i in range(len(y_train))]), 
#             torch.cat([y_train[i][2].unfold(dimension = 0, size = args.window_size, step = args.window_step).permute(0, 2, 1) for i in range(len(y_train))]), 
#             torch.cat([y_train[i][3].unfold(dimension = 0, size = args.window_size, step = args.window_step).permute(0, 2, 1) for i in range(len(y_train))])]}, 
#             './data/data_train_wmx.pt')
# torch.save({"X": torch.cat([X_val[i].unfold(dimension = 0, size = args.window_size, step = args.window_step).permute(0, 2, 1) for i in range(len(X_val))], dim = 0), 
#         'y': [torch.cat([y_val[i][0].unfold(dimension = 0, size = args.window_size, step = args.window_step).permute(0, 3, 1, 2) for i in range(len(y_val))]), 
#             torch.cat([y_val[i][1].squeeze(1).repeat(y_val[i][0].shape[0], 1).unfold(dimension = 0, size = args.window_size, step = args.window_step).permute(0, 2, 1) for i in range(len(y_val))]), 
#             torch.cat([y_val[i][2].unfold(dimension = 0, size = args.window_size, step = args.window_step).permute(0, 2, 1) for i in range(len(y_val))]), 
#             torch.cat([y_val[i][3].unfold(dimension = 0, size = args.window_size, step = args.window_step).permute(0, 2, 1) for i in range(len(y_val))])]}, 
#             './data/data_val_wmx.pt')

# 配置params

In [ ]:
from utils.wmx_tools import create_args

create_args("./configs/train_params_LSTM_Temp.yaml")

# 生成网络config

In [ ]:
from utils.TransPose.config import paths, joint_set, acc_scale, vel_scale
import yaml

# 生成config
with open("configs/models/LSTM_Small.yaml", "w", encoding = "utf-8") as f:
    temp_write = {}
    temp_write["pose_net"] = {
        "net_name" : "WMX_LSTMBlock",
        "input_size" : 36,
        "hidden_size" : 32,
        "output_size" : joint_set.n_full * 3,
        "num_layers" : 2,
    }
    temp_write["vel_net"] = {
        "net_name" : "WMX_LSTMBlock",
        "input_size" : joint_set.n_full * 3 + 9,
        "hidden_size" : 32,
        "output_size" : 3,
        "num_layers" : 2,
    }
    temp_write["contact_net"] = {
        "net_name" : "WMX_LSTMBlock",
        "input_size" : joint_set.n_full * 3 + 9 + 3,
        "hidden_size" : 32,
        "output_size" : 2,
        "num_layers" : 2,
    }
    yaml.dump(temp_write, f)

In [ ]:
from utils.TransPose.config import paths, joint_set, acc_scale, vel_scale
import yaml

# 生成config
with open("configs/models/TEA_Small.yaml", "w", encoding = "utf-8") as f:
    temp_write = {}
    temp_write["pose_net"] = {
        "net_name" : "WMX_TemporalExternalAttentionBlock",
        "input_size" : 36,
        "hidden_size" : 32,
        "output_size" : joint_set.n_full * 3,
        "num_layers" : 4,
    }
    temp_write["vel_net"] = {
        "net_name" : "WMX_TemporalExternalAttentionBlock",
        "input_size" : joint_set.n_full * 3 + 9,
        "hidden_size" : 32,
        "output_size" : 3,
        "num_layers" : 4,
    }
    temp_write["contact_net"] = {
        "net_name" : "WMX_TemporalExternalAttentionBlock",
        "input_size" : joint_set.n_full * 3 + 9 + 3,
        "hidden_size" : 32,
        "output_size" : 2,
        "num_layers" : 4,
    }
    yaml.dump(temp_write, f)

In [ ]:
from utils.TransPose.config import paths, joint_set, acc_scale, vel_scale
import yaml

# 生成config
with open("configs/models/SegNext_Small.yaml", "w", encoding = "utf-8") as f:
    temp_write = {}
    temp_write["pose_net"] = {
        "net_name" : "WMX_SegNextBlock",
        "input_size" : 36,
        "hidden_size" : 32,
        "output_size" : joint_set.n_full * 3,
        "num_layers" : 4,
    }
    temp_write["vel_net"] = {
        "net_name" : "WMX_SegNextBlock",
        "input_size" : joint_set.n_full * 3 + 9,
        "hidden_size" : 32,
        "output_size" : 3,
        "num_layers" : 4,
    }
    temp_write["contact_net"] = {
        "net_name" : "WMX_SegNextBlock",
        "input_size" : joint_set.n_full * 3 + 9 + 3,
        "hidden_size" : 32,
        "output_size" : 2,
        "num_layers" : 4,
    }
    yaml.dump(temp_write, f)